In [1]:
### Gonna use the following genres

# Electronic ---------------> 15
# Rock ---------------------> 12
# Pop ----------------------> 10
# Folk ---------------------> 17
# Hip-Hop + Hip-Hop Beats --> 21 + 811
# Blues + Jazz -------------> 4 + 74

In [2]:
from glob import glob

import pandas as pd

import re

In [3]:
raw_tracks_pd = pd.read_csv(
  "./data/fma_metadata/raw_tracks.csv", usecols=["track_id", "track_genres"]
)

In [4]:
raw_tracks_pd = raw_tracks_pd[["track_id", "track_genres"]]

In [5]:
raw_tracks_pd["genre_list"] = raw_tracks_pd.track_genres.apply(
  lambda row: re.findall(r'\d+', str(row))
)

In [6]:
# genres_pd = pd.read_csv(
#   "./data/fma_metadata/genres.csv", 
#   usecols=["genre_id", "title", "top_level", "parent"]
# )

In [7]:
# genres_id_unique = genres_pd.top_level.unique()
# genres_pd[genres_pd["genre_id"].isin(genres_id_unique)]

In [8]:
raw_tracks_pd = raw_tracks_pd[["track_id", "genre_list"]]

In [9]:
raw_tracks_pd

,track_id,genre_list
0,2,[21]
1,3,[21]
2,5,[21]
3,10,[10]
4,20,"[76, 103]"
...,...,...
109722,155316,[25]
109723,155317,[25]
109724,155318,[25]
109725,155319,[25]


In [10]:
num_no_genre = 0

for i in range(len(raw_tracks_pd.index)):
  
  try:
    raw_tracks_pd.iloc[i]["genre_list"][0]
    pass
  
  except:
    num_no_genre += 1
    # print(raw_tracks_pd.iloc[i]["genre_list"])

In [11]:
num_no_genre

2609

In [12]:
raw_tracks_clean_pd = raw_tracks_pd[
  raw_tracks_pd['genre_list'].map(lambda d: len(d)) > 0
]


In [13]:
raw_tracks_clean_pd

,track_id,genre_list
0,2,[21]
1,3,[21]
2,5,[21]
3,10,[10]
4,20,"[76, 103]"
...,...,...
109722,155316,[25]
109723,155317,[25]
109724,155318,[25]
109725,155319,[25]


In [14]:
num_no_genre = 0

for i in range(len(raw_tracks_clean_pd.index)):
  
  try:
    raw_tracks_clean_pd.iloc[i]["genre_list"][0]
    pass
  
  except:
    num_no_genre += 1

In [15]:
num_no_genre

0

## From sub-genre to macro-genre ##

In [16]:
genres_pd = pd.read_csv(
  "./data/fma_metadata/genres.csv", 
  usecols=["genre_id", "top_level"]
)

In [17]:
genres_pd

,genre_id,top_level
0,1,38
1,2,2
2,3,3
3,4,4
4,5,5
...,...,...
158,1032,2
159,1060,2
160,1156,2
161,1193,38


In [18]:
sub_genre_to_macro

NameError: name 'sub_genre_to_macro' is not defined

In [ ]:
sub_genre_to_macro = {}

for index, row in genres_pd.iterrows():
    
  sub_genre_to_macro[str(row.genre_id)] = str(row.top_level)

In [ ]:
raw_tracks_clean_pd["genre"] = raw_tracks_clean_pd.genre_list.apply(
  # lambda g: g[0]
  lambda g: sub_genre_to_macro[g[0]]
)

In [ ]:
raw_tracks_clean_pd

,track_id,genre_list,genre
0,2,[21],21
1,3,[21],21
2,5,[21],21
3,10,[10],10
4,20,"[76, 103]",10
...,...,...,...
109722,155316,[25],12
109723,155317,[25],12
109724,155318,[25],12
109725,155319,[25],12


In [ ]:
GENRES_LIST = ["15", "12", "10", "17", "21", "811", "4", "74", "3"]

raw_tracks_filtered_pd = raw_tracks_clean_pd[
  raw_tracks_clean_pd["genre"].isin(GENRES_LIST)
]

In [ ]:
genre_id_to_str = {
  '21': "Hip-Hop", 
  '10': "Pop", 
  '12': "Rock", 
  '17': "Folk", 
  '4': "Jazz", 
  '3': "Jazz",
  '15': "Electronic"
}

raw_tracks_filtered_pd["genre"] = raw_tracks_filtered_pd.genre.apply(
  lambda g: genre_id_to_str[g]
)

In [ ]:
raw_tracks_filtered_pd.groupby("genre").count()

,track_id,genre_list
genre,,
Classical,2106,2106
Electronic,25371,25371
Folk,6274,6274
Hip-Hop,4619,4619
Jazz,4046,4046
Pop,8430,8430
Rock,22036,22036


In [ ]:
downsample_pd = raw_tracks_filtered_pd.groupby('genre').apply(
  lambda x : x.head(4046).reset_index(drop=True)
)[["track_id", "genre"]].reset_index(drop=True)


In [ ]:
downsample_pd.groupby("genre").count()

,track_id
genre,
Electronic,4046
Folk,4046
Hip-Hop,4046
Jazz,4046
Pop,4046
Rock,4046
